<h1><center>Une application de recommandation de contenu</center></h1>

<hr>
<h3><center>Élaboration d’un modèle de type Collaborative Filtering</center></h3>
<br>

>__Réalisé par : Said Arrazouaki__


>__Encadré par : Addi Ait-Mlouk__

## Introduction

Après avoir exploré les approches basées sur le contenu, ce troisième notebook aborde une technique complémentaire : le filtrage collaboratif. Cette méthode ne se focalise plus sur les caractéristiques intrinsèques des articles, mais sur les interactions entre les utilisateurs et les contenus : l’idée est que des utilisateurs ayant réagi de manière similaire à certains articles auront sans doute des goûts proches. En analysant la matrice des clics et en la factorisant en vecteurs latents représentant d’un côté les utilisateurs et de l’autre les articles, nous chercherons à prédire quels contenus pourraient plaire à chaque lecteur, même s’il ne les a encore jamais consultés. Nous décrirons ici la construction et l’évaluation d’un tel modèle collaboratif. 

## Chargement des données 

Commençons par charger les différentes données nécessaires :
- Le fichier articles_embeddings_reduced.pickle
- Le fichier articles_metadata.csv
- Les fichiers clicks/clicks_hour_*.csv

In [1]:
import pandas as pd
import pickle
import glob
from sklearn.metrics.pairwise import cosine_similarity 
import numpy as np  
from joblib import Parallel, delayed 

In [2]:
# Charger les métadonnées des articles
articles_df = pd.read_csv('../data/articles_metadata.csv')
print("Nombre d'articles :", len(articles_df))

Nombre d'articles : 364047


In [3]:
# Charger la matrice d'embeddings des articles
with open('../data/processed/articles_embeddings_reduced.pickle', 'rb') as f:
    embeddings_matrix = pickle.load(f)
print("Taille de la matrice d'embeddings :", embeddings_matrix.shape)

Taille de la matrice d'embeddings : (364047, 75)


In [4]:
# Charger et concaténer tous les fichiers de clics horaires
clicks_files = glob.glob('../data/clicks/clicks_hour_*.csv')
clicks_list = [pd.read_csv(f) for f in clicks_files]
clicks_df = pd.concat(clicks_list, ignore_index=True)
print("Nombre total de clics :", len(clicks_df))

Nombre total de clics : 2988181


In [5]:
# Lire les user_clicks
with open('../data/processed/user_clicks.pickle', 'rb') as f:
    user_clicks = pickle.load(f)

In [6]:
# Articles populaires pour cold start et optimisation
popular_articles = clicks_df['click_article_id'].value_counts().head(1000).index.tolist()

In [7]:
# enregistrer les articles populaires
with open('../data/processed/popular_articles.pickle', 'wb') as f:
    pickle.dump(popular_articles, f)

## Construction de la matrice utilisateur-article

Cette partie prépare les interactions utilisateurs-articles en calculant des ratings implicites pondérés. Les données sont formatées pour l'algorithme SVD avec la bibliothèque Surprise.

In [33]:
from surprise import Dataset, Reader, accuracy, SVD
from surprise.model_selection import train_test_split

In [34]:
# Préparer les données avec pondération (rating = count / session_size)
interactions = clicks_df.groupby(['user_id', 'click_article_id', 'session_size']).size().reset_index(name='count')
interactions['rating'] = interactions['count'] / interactions['session_size']  # Pondération
print("Nombre de couples user-article :", len(interactions))

Nombre de couples user-article : 2977178


In [35]:
# Définir un reader surprise
reader = Reader(rating_scale=(1, interactions['count'].max()))
data = Dataset.load_from_df(interactions[['user_id', 'click_article_id', 'rating']], reader)

# Split en train/test
trainset, testset = train_test_split(data, test_size=0.2)

## Entraînement du modèle SVD sur les données implicites

Nous entraînons ici un modèle SVD pour prédire les préférences des utilisateurs. Le modèle est évalué avec RMSE et Precision@5, puis sauvegardé pour une utilisation en production.

In [36]:
# Entraînement SVD
algo = SVD(n_factors=50, n_epochs=20, random_state=42)
algo.fit(trainset)

In [37]:
# Sauvegarde du modèle pour production
with open('../data/processed/svd_model.pickle', 'wb') as f:
    pickle.dump(algo, f)

In [38]:
# Évaluation RMSE
predictions = algo.test(testset)
print("RMSE :", accuracy.rmse(predictions))

# Évaluation Precision@5 
def precision_at_k(predictions, k=5):
    user_est_true = {}
    for uid, _, true_r, est, _ in predictions:
        if uid not in user_est_true:
            user_est_true[uid] = []
        user_est_true[uid].append((est, true_r))
    precisions = []
    for uid, user_ratings in user_est_true.items():
        user_ratings.sort(key=lambda x: x[0], reverse=True)
        n_rel = sum((true_r > 0) for (_, true_r) in user_ratings[:k])
        precisions.append(n_rel / k if k > 0 else 0)
    return sum(precisions) / len(precisions)

print("Precision@5 :", precision_at_k(predictions))

RMSE: 0.6645
RMSE : 0.6644914367615806
Precision@5 : 0.45312426977838055


## Génération des recommandations avec le modèle SVD

Cette section définit les fonctions pour générer des recommandations, incluant une approche Collaborative flexible (tous articles ou populaires)

In [39]:
# Liste de tous les articles
all_article_ids = articles_df['article_id'].unique().tolist()

def recommend_articles_collaborative(user_id, top_n=5, candidate_articles=popular_articles):
    if candidate_articles is None:
        candidate_articles = all_article_ids
        
    try:
        inner_uid = algo.trainset.to_inner_uid(user_id)  # noqa: F841
    except ValueError:  
        return popular_articles[:top_n]  

    seen_articles = set(user_clicks.get(user_id, []))
    recommendations = []
    for art_id in candidate_articles:  # Limite aux candidats pour perf
        if art_id in seen_articles:
            continue
        pred = algo.predict(user_id, art_id)
        recommendations.append((art_id, pred.est))
    recommendations.sort(key=lambda x: x[1], reverse=True)
    return [art for art, _ in recommendations[:top_n]] or popular_articles[:top_n]


In [40]:
# Exemple
example_user = list(user_clicks.keys())[0]
print("Articles lus par", example_user, ":", user_clicks[example_user][:5])
print("Recos collab :", recommend_articles_collaborative(example_user))

Articles lus par 0 : [157541, 68866, 96755, 313996, 160158]
Recos collab : [160974, 272143, 336221, 234698, 123909]


In [42]:
# Précalcul optimisé avec parallélisme (limité à 100 users pour test)
users_sample = list(user_clicks.keys())[:100]  # Augmente en prod
def compute_recos(user):
    return user, recommend_articles_collaborative(user)

top_recos = dict(Parallel(n_jobs=-1)(delayed(compute_recos)(user) for user in users_sample))
with open('../data/processed/precomputed_recos_collab.pickle', 'wb') as f:
    pickle.dump(top_recos, f)

## Hybride

Cette section introduite une version hybride combinant Collaborative et Content-Based

In [ ]:
# Fonction hybride (Collab + Content-Based)
def recommend_hybrid(user_id, top_n=5):
    collab_recos = recommend_articles_collaborative(user_id, top_n=10)  # Plus pour mélange
    if user_id not in user_clicks:
        return collab_recos[:10]
    last_article = user_clicks[user_id][-1]  # Dernier cliqué
    last_emb = embeddings_matrix.iloc[last_article]
    sims = cosine_similarity([last_emb], embeddings_matrix)[0]
    content_recos = np.argsort(sims)[-11:-1][::-1]  # Top 10 similaires, exclure soi-même
    hybrid = list(set(collab_recos + list(content_recos)))[:top_n]  
    return collab_recos, content_recos, hybrid

In [47]:
# Exemple
example_user = list(user_clicks.keys())[0]
collab_recos, content_recos, hybrid = recommend_hybrid(example_user)
print("Articles lus par", example_user, ":", user_clicks[example_user][:5])
print("Collaboratif recommendations :", collab_recos)
print("Content-based recommendations:", content_recos)
print("Hybride recommendations :", hybrid)

Articles lus par 0 : [157541, 68866, 96755, 313996, 160158]
Collaboratif recommendations : [160974, 272143, 336221, 234698, 123909, 336223, 96210, 162655, 183176, 168623]
Content-based recommendations: [86143 87427 87676 87278 86613 87670 85792 87030 86426 85768]
Hybride recommendations : [87427, 123909, 183176, 85768, 272143]


## Conclusion

Ce notebook a mis en œuvre un système de recommandation basé sur le Collaborative Filtering avec l'algorithme SVD, optimisé pour la performance et la scalabilité. Nous avons corrigé les lenteurs initiales en limitant les prédictions aux articles populaires (optionnel), ajouté une pondération des ratings implicites, et intégré une approche hybride utilisant les embeddings pour gérer les nouveaux articles. L'évaluation avec RMSE et Precision@5 montre des résultats prometteurs, et le précalcul des recommandations facilite l'intégration dans Azure Functions via Blob Storage. Pour la suite, un réentraînement périodique du modèle et une combinaison avec le Content-Based Filtering permettront d'améliorer la couverture des nouveaux utilisateurs et articles, répondant ainsi aux besoins de la startup My Content.blablabla